In [1]:
#Import Dependencies

import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
#Extract of ETL begins here

In [3]:
#Import the three NBA CSV Files

players_csv = "players.csv"
player_data_csv = "player_data.csv"
season_stats_csv = "Seasons_stats.csv"

df_players = pd.read_csv(players_csv)
df_player_data = pd.read_csv(player_data_csv)
df_season_stats = pd.read_csv(season_stats_csv)

In [4]:
#Read Header
df_players.head()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [5]:
#Read Header
df_player_data.head()
#note, player_data.csv file has the same information as the players.csv file and more, so players.csv file
#can be ignored

,name,year_start,year_end,position,height_feet,height_inches,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6.0,10.0,240.0,6/24/1968,Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6.0,9.0,235.0,4/7/1946,Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7.0,2.0,225.0,4/16/1947,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6.0,1.0,162.0,3/9/1969,Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6.0,6.0,223.0,11/3/1974,San Jose State University


In [6]:
#Read Dataframe
df_season_stats.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [7]:
#Transform - Begins
#Delete rows that have NULL values in all columns in the three dataframes
#Players.csv was noted to not have data for all players, and it also contained the same information
#as Players_data, so it was ignored for the rest of the analysis
df_clean_player_data = df_player_data.dropna(how='all')
df_season_stats = df_season_stats.dropna(how ="all")

#Database Schema and ERD were created using QuickDBD based on column headers, 
#and database/tables were created in PostGres

In [8]:
#get column names
df_player_data.columns

Index(['name', 'year_start', 'year_end', 'position', 'height_feet',
       'height_inches', 'weight', 'birth_date', 'college'],
      dtype='object')

In [9]:
#make filtered player_data dataframe dropping the "year_start" and "year_End" column to push into the players_data table in the database
cols_2 = ['name','birth_date', 'height_feet',
       'height_inches', 'weight', 'college']
df_player_data_2= df_clean_player_data[cols_2].copy()
df_player_data_2.head()

,name,birth_date,height_feet,height_inches,weight,college
0,Alaa Abdelnaby,6/24/1968,6.0,10.0,240.0,Duke University
1,Zaid Abdul-Aziz,4/7/1946,6.0,9.0,235.0,Iowa State University
2,Kareem Abdul-Jabbar,4/16/1947,7.0,2.0,225.0,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,3/9/1969,6.0,1.0,162.0,Louisiana State University
4,Tariq Abdul-Wahad,11/3/1974,6.0,6.0,223.0,San Jose State University


In [23]:
#make dataframe using name and year_start/end for the "Players" table in the database to distinguish 
#between players with the same names but different birthdays
cols = ["name", "year_start", "year_end"]
player_info = df_player_data[cols].copy()
player_info.head()

,name,year_start,year_end
0,Alaa Abdelnaby,1991,1995
1,Zaid Abdul-Aziz,1969,1978
2,Kareem Abdul-Jabbar,1970,1989
3,Mahmoud Abdul-Rauf,1991,2001
4,Tariq Abdul-Wahad,1998,2003


In [11]:
df_season_stats.columns

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [12]:
cols_1 = ['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
        'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
df_season_stats_2 = df_season_stats[cols_1].copy()
df_season_stats_2 = df_season_stats_2.dropna(how ="all")

df_season_stats_2.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,144.0,516.0,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,102.0,274.0,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,174.0,499.0,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,22.0,86.0,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,21.0,82.0,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [13]:
df_season_stats_3= df_season_stats_2.rename(columns={"Pos": "Position",
                                                    "FG%": "FG_percent",
                                                    "3P%":"3P_percent",
                                                    "2P%": "2P_percent",
                                                    "eFG%": "eFG_percent",
                                                    "FT%": "FT_percent",
                                                    "Tm": "Team"})
df_season_stats_3.count()

Year           24624
Player         24624
Position       24624
Age            24616
Team           24624
G              24624
GS             18233
MP             24138
FG             24624
FGA            24624
FG_percent     24525
3P             18927
3PA            18927
3P_percent     15416
2P             24624
2PA            24624
2P_percent     24496
eFG_percent    24525
FT             24624
FTA            24624
FT_percent     23766
ORB            20797
DRB            20797
TRB            24312
AST            24624
STL            20797
BLK            20797
TOV            19645
PF             24624
PTS            24624
dtype: int64

In [14]:
#Create Tables for "unique" Position, Team and College

position = df_season_stats.Pos.unique()
Pos_df = pd.DataFrame(position, columns =["position"]).dropna()
Pos_df

teams = df_season_stats.Tm.unique()
team_df = pd.DataFrame(teams, columns = ["team"]).dropna()
team_df

colleges = df_player_data.college.unique()
colleges_2 = pd.DataFrame(colleges)
colleges_3 = colleges_2.rename(columns={0: "college"}).dropna()

In [15]:
#connect to the specific database in PostGres containing the NBA data tables
#password must be replaced with actual password for the database connection to work
connection_string = "postgres:{password}@localhost:5432/NBA_data"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
#Check Table Name
engine.table_names()


['Teams', 'Positions', 'Players', 'Player_data', 'Colleges', 'SeasonStats']

In [ ]:
#Load of ETL begins here

In [19]:
#Push colleges_3 df to the  Colleges Table in PostGres
colleges_3.to_sql(name="Colleges", con=engine, if_exists='append', index=False)


In [20]:
#Push teams df to the  Teams Table in PostGres
team_df.to_sql(name="Teams", con=engine, if_exists='append', index=False)

In [21]:
#push unique positions df to the Positions table in PostGres
Pos_df.to_sql(name="Positions", con=engine, if_exists='append', index=False)

In [24]:
#push player df to the Players table in PostGres
player_info.to_sql(name="Players", con=engine, if_exists='append', index=False)

In [25]:
#push player_data_2 df to the Player_data table in PostGres
df_player_data_2.to_sql(name="Player_data", con=engine, if_exists='append', index=False)

In [26]:
#import season stats table
df_season_stats_3.to_sql(name="SeasonStats", con=engine, if_exists='append', index=False)

In [28]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "Players"', con=engine).head()

,name,year_start,year_end
0,Alaa Abdelnaby,1991,1995
1,Zaid Abdul-Aziz,1969,1978
2,Kareem Abdul-Jabbar,1970,1989
3,Mahmoud Abdul-Rauf,1991,2001
4,Tariq Abdul-Wahad,1998,2003


In [29]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "Player_data"', con=engine).head()

,name,birth_date,height_feet,height_inches,weight,college
0,Alaa Abdelnaby,6/24/1968,6.0,10.0,240.0,Duke University
1,Zaid Abdul-Aziz,4/7/1946,6.0,9.0,235.0,Iowa State University
2,Kareem Abdul-Jabbar,4/16/1947,7.0,2.0,225.0,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,3/9/1969,6.0,1.0,162.0,Louisiana State University
4,Tariq Abdul-Wahad,11/3/1974,6.0,6.0,223.0,San Jose State University


In [30]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "Teams"', con=engine).head()

,id,team
0,1,FTW
1,2,INO
2,3,CHS
3,4,TOT
4,5,DNN


In [31]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "Colleges"', con=engine).head()

,id,college
0,1,Duke University
1,2,Iowa State University
2,3,"University of California, Los Angeles"
3,4,Louisiana State University
4,5,San Jose State University


In [32]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "Positions"', con=engine).head()

,id,position
0,1,G-F
1,2,SG
2,3,SF
3,4,F
4,5,G


In [33]:
#confirm data loaded to SQL 
pd.read_sql_query('select * from "SeasonStats"', con=engine).head()

,Year,Player,Position,Age,Team,G,GS,MP,FG,FGA,...,FT_percent,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,144.0,516.0,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,102.0,274.0,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,174.0,499.0,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,22.0,86.0,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,21.0,82.0,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
